In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
import joblib
import requests
import os

In [3]:


url = 'http://www.bom.gov.au/climate/dwo/IDCJDW2801.latest.shtml'

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

dfs = pd.read_html(r.text)
dfs[0]

Date  \
                                                 Date   
                                                 Date   
0                                                   1   
1                                                   2   
2                                                   3   
3                                                   4   
4                                                   5   
5                                                   6   
6                                                   7   
7                                                   8   
8                                                   9   
9                                                  10   
10                                                 11   
11                                                 12   
12                                                 13   
13                                                 14   
14                                                 15   
15                                                 16   
16                                                 17   
17                                                 18   
18                                                 19   
19                                                 20   
20                                                 21   
21                                                 22   
22                                                 23   
23                                                 24   
24                                                 25   
25                                                 26   
26                                                 27   
27                                                 28   
28  Statistics for the first 28 days of November 2022   
29                                               Mean   
30                                             Lowest   
31                                            Highest   
32                                              Total   

                                                  Day  \
                                                  Day   
                                                  Day   
0                                                  Tu   
1                                                  We   
2                                                  Th   
3                                                  Fr   
4                                                  Sa   
5                                                  Su   
6                                                  Mo   
7                                                  Tu   
8                                                  We   
9                                                  Th   
10                                                 Fr   
11                                                 Sa   
12                                                 Su   
13                                                 Mo   
14                                                 Tu   
15                                                 We   
16                                                 Th   
17                                                 Fr   
18                                                 Sa   
19                                                 Su   
20                                                 Mo   
21                                                 Tu   
22                                                 We   
23                                                 Th   
24                                                 Fr   
25                                                 Sa   
26                                                 Su   
27                                                 Mo   
28  Statistics for the first 28 days of November 2022   
29                                               Mean   
30                                             Lowest   
31                                            Highest   
3

In [4]:
df=dfs[0]
df1=df.iloc[:26]

In [5]:
loca=["Canberra"]*26
Month=["11"]*26
df1.insert(2, "Location", loca)
df1.insert(3, "Month", Month)

In [7]:
pd.set_option("display.max_columns", None)
df1

Date Day  Location Month Temps        Rain Evap   Sun Max wind gust       \
   Date Day                   Min   Max  Rain Evap   Sun           Dir  Spd   
   Date Day                    °C    °C    mm   mm hours          km/h km/h   
0     1  Tu  Canberra    11   8.5  13.9  40.4  NaN   NaN           NNW   80   
1     2  We  Canberra    11   4.3  13.3   3.2  NaN   NaN           WNW   52   
2     3  Th  Canberra    11   5.5  17.2     0  NaN   NaN             W   41   
3     4  Fr  Canberra    11   6.3  19.7     0  NaN   NaN             E   30   
4     5  Sa  Canberra    11  10.6  20.7     0  NaN   NaN             N   24   
5     6  Su  Canberra    11   6.2  23.2     0  NaN   NaN             W   33   
6     7  Mo  Canberra    11   8.7  23.3     0  NaN   NaN           SSW   48   
7     8  Tu  Canberra    11   7.6  23.0   9.0  NaN   NaN             N   28   
8     9  We  Canberra    11   9.0  24.0   0.2  NaN   NaN             E   35   
9    10  Th  Canberra    11   8.2  23.0     0  NaN   NaN            NE   37   
10   11  Fr  Canberra    11   9.4  22.0     0  NaN   NaN           NNW   35   
11   12  Sa  Canberra    11  10.5  26.8     0  NaN   NaN             E   50   
12   13  Su  Canberra    11  14.8  23.3  14.4  NaN   NaN           NNW   50   
13   14  Mo  Canberra    11  11.9  19.0  21.0  NaN   NaN           WNW   65   
14   15  Tu  Canberra    11  10.1  17.0   1.2  NaN   NaN            NW   50   
15   16  We  Canberra    11   3.0  14.5     0  NaN   NaN           WSW   48   
16   17  Th  Canberra    11   3.3  18.2   0.6  NaN   NaN             S   35   
17   18  Fr  Canberra    11   2.6  21.1     0  NaN   NaN           NNE   30   
18   19  Sa  Canberra    11   7.8  22.4     0  NaN   NaN            NW   39   
19   20  Su  Canberra    11  11.7  18.3   6.2  NaN   NaN           WNW   70   
20   21  Mo  Canberra    11   8.3  12.9   2.2  NaN   NaN           WNW   74   
21   22  Tu  Canberra    11   3.6   NaN   1.8  NaN   NaN           NaN  NaN   
22   23  We  Canberra    11   9.0  19.5     0  NaN   NaN            NW   50   
23   24  Th  Canberra    11   4.4  22.9     0  NaN   NaN           NNW   48   
24   25  Fr  Canberra    11   5.0  24.0     0  NaN   NaN           WNW   50   
25   26  Sa  Canberra    11   5.6  24.0     0  NaN   NaN             N   37   

           9 am                              3 pm                             
     Time  Temp   RH  Cld  Dir  Spd    MSLP  Temp  RH  Cld  Dir  Spd    MSLP  
    local    °C    %  8th km/h km/h     hPa    °C   %  8th km/h km/h     hPa  
0   23:13  10.0   84    8   NW   31   991.8  11.2  69    7   NW   28   992.0  
1   12:31   7.7   62    1  WNW   28  1001.8  11.7  47    1  WNW   33  1005.7  
2   13:24  11.1   67    4   NW   19  1014.5  15.2  49    8   NW   22  1015.2  
3   19:34  11.5   71  NaN    W    9  1025.8  18.1  45    8   NW   11  1024.4  
4   14:54  13.8   69    8  ESE   13  1028.3  19.7  43    6    N   11  1024.9  
5   12:11  15.2   75  NaN  SSE    7  1025.4  22.2  39    1    W   17  1021.5  
6   13:37  16.2   66  NaN    S    7  1022.2  14.5  81    8  SSW   28  1020.0  
7   14:42  15.1   72  NaN  SSW    9  1021.0  22.0  44    1  WNW    7  1018.0  
8   16:00  15.7   73  NaN   SW    7  1022.0  22.9  46    8   NE   17  1018.8  
9   10:27  14.2   65  NaN   NE    9  1021.9  22.1  37  NaN  NNE   13  1017.1  
10  10:51  17.1   70    8    N    9  1013.9  19.8  69    8   NW   15  1011.4  
11  17:58  19.6   72  NaN   SW    2  1012.0  25.8  42    3  NNW   20  1009.6  
12  10:40  15.7  100    8  ENE   11  1008.6  21.2  75    8   NW   17  1003.9  
13  09:51  15.2   67    8   NW   41   999.8  16.6  56    8  WNW   39  1001.1  
14  13:59  13.6   64    1  NNW   20  1007.4  14.6  45    6   NW   35  1007.0  
15  08:57   9.2   62    8  WNW   17  1009.1  12.7  41  NaN    W   17  1009.1  
16  08:51   9.8   52  NaN    S   20  1014.3  16.8  36  NaN    S   15  1012.3  
17  15:50  12.2   59  NaN  SSE    7  1015.8  19.8  36    1   NE   13  1011.3  
18  15:10  13.2   75    8  SSE    4  1009.9  21.0  5

In [8]:
for i in df.columns:
    print(i)

('Date', 'Date', 'Date')
('Day', 'Day', 'Day')
('Temps', 'Min', '°C')
('Temps', 'Max', '°C')
('Rain', 'Rain', 'mm')
('Evap', 'Evap', 'mm')
('Sun', 'Sun', 'hours')
('Max wind gust', 'Dir', 'km/h')
('Max wind gust', 'Spd', 'km/h')
('Max wind gust', 'Time', 'local')
('9 am', 'Temp', '°C')
('9 am', 'RH', '%')
('9 am', 'Cld', '8th')
('9 am', 'Dir', 'km/h')
('9 am', 'Spd', 'km/h')
('9 am', 'MSLP', 'hPa')
('3 pm', 'Temp', '°C')
('3 pm', 'RH', '%')
('3 pm', 'Cld', '8th')
('3 pm', 'Dir', 'km/h')
('3 pm', 'Spd', 'km/h')
('3 pm', 'MSLP', 'hPa')


In [9]:
df.to_csv("rainfalltest.csv")

In [51]:
df2 = df.set_axis(['V', 'W', 'X', 'Y', 'Z'], axis=1, inplace=False)

ValueError: Length mismatch: Expected axis has 22 elements, new values have 5 elements